문제: m × n의 크기를 가지는 2차원 배열 I를 입력 받아, 연결된 구성요소를 구분할 수 있는 번호가 부여된 2차원 배열 L과 연결된 구성요소의 수 c를 출력하라.  
단, I[y][x]의 값은 0 또는 1.

입력: 정수 m, n; 이진 2차원 배열 I[n][m];  
출력: 정수 c; 정수 2차원 배열L[n][m]; // 0으로 초기화 필요





In [ ]:
import numpy as np
import sys

def read_neighbor8(y, x, bImage):
    #주변 8방향 이웃 값 가져옴
    neighbor8 = [0 for _ in range(8)]
    dx = [1,1,0,-1,-1,-1,0,1]
    dy = [0,1,1,1,0,-1,-1,-1]
    for i in range(8):
        nx = x + dx[i]
        ny = y + dy[i]        
        neighbor8[i] = bImage[ny][nx]
    
    return neighbor8


def cal_coord(i, y, x):
    #방향을 통해 현재 좌표 계산
    dx = [1,1,0,-1,-1,-1,0,1]
    dy = [0,1,1,1,0,-1,-1,-1]
    nx = x + dx[i]
    ny = y + dy[i]
    return ny, nx


def BTracing8(y, x, label, tag, labImage, bImage):
    if tag == "FOREWARD": 
        cur_orient = 0 #시작조건일 경우 방향 0
    else: 
        cur_orient = 6 #hole 일 경우 방향 6

    end_x, end_y = x, y #시작 위치 저장
    

    while True:
        neighbor8 = read_neighbor8(y, x, bImage) #8방향 이웃한 값들 읽어옴
        
        start_o = (8 + cur_orient - 2) % 8 #시계방향으로 
        i = 0
        while i < 8: #8방향 동안
            add_o = (start_o + i) % 8 #현재 방향에서 이웃이 객체일 경우 break
            if neighbor8[add_o] != 0:
                break
            i += 1

        if i < 8: #객체인 경우
            cury,curx = y,x #방향 바꾸기 전 좌표
            y, x = cal_coord(add_o, y, x) #해당 위치 좌표 받아옴
            cur_orient = add_o #현재 방향 업데이트
            if labImage[y][x] <= 0: #객체인데 레이블링이 안되어있을 경우
                labImage[y][x] = label
        
            elif labImage[y][x] > 0 and labImage[y][x] < label: #레이블링 번호가 현재 레이블 번호보다 작을 경우
                if bImage[cury][curx] == 1:
                    label = labImage[y][x] #라벨을 더 낮은 번호로 바꿔주고
                    labImage[cury][curx] = label #방향 전환 전 레이블도 바꿔줌
                    
        else:
            break #객체가 아닌경우

        if y == end_y and x == end_x: #시작위치로 돌아왔을 경우
            break

    return labImage


def LabelingwithBT(bImage): #레이블링 함수
    HEIGHT, WIDTH = len(bImage), len(bImage[0]) 
    labImage = [[0 for _ in range(WIDTH)] for _ in range(HEIGHT)] #레이블링 결과 리스트

    label_number = 1 #레이블 넘버 2부터 시작
    for i in range(1, HEIGHT - 1):
        for j in range(1, WIDTH - 1):
            if labImage[i][j] <= 0: #레이블링이 되어있지 않고
                cur_p = bImage[i][j] 
                if cur_p == 1:  # 현재 위치가 객체일때
                    ref_p1 = labImage[i][j - 1] #왼쪽
                    ref_p2 = labImage[i - 1][j] #왼쪽 대각선
                    if ref_p1 >= 2:  #왼쪽이 레이블링 되어있으면 해당 레이블 값 대입
                        labImage[i][j] = ref_p1
                    elif ref_p1 == 0 and ref_p2 >= 2:  # hole일 경우
                        labImage[i][j] = ref_p2 #왼쪽 대각선 레이블 값 입력 후
                        labImage = BTracing8(i, j, ref_p2, "BACKWARD", labImage, bImage) #8방향 살펴봄
                    elif ref_p1 == 0 and ref_p2 == 0: #시작위치일 경우
                        label_number += 1 #레이블 넘버 + 1
                        labImage[i][j] = label_number #해당 위치에 레이블 넘버 대입
                        labImage = BTracing8(i, j, label_number, "FOREWARD", labImage, bImage) #8방향 살펴봄
                else:
                    labImage[i][j] = 0  #객체가 아닐때

    return labImage



sys.stdin = open('./input.txt','r') 
T = int(sys.stdin.readline()) #테스트 케이스 개수

for t in range(T):
    N,M = map(int, sys.stdin.readline().split()) #행, 열
    bImage = [ list(map(int, sys.stdin.readline().split())) for _ in range(N)] #레이블링 할 이진 리스트
    result = LabelingwithBT(bImage) #레이블링 결과
    
    for i in range(N):
        print(bImage[i]) #입력
    print()
    
    for i in range(N):
        print(result[i]) #출력
    print()
